Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/mads_thesis')
!pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/mads_thesis


Load libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, roc_auc_score

Load and format data

In [ ]:
train_data = pd.read_pickle('Video_Games_final_train.pkl.gz')
test_data = pd.read_pickle('Video_Games_final_test.pkl.gz')

labels = {'Unhelpful': 0, 'Helpful': 1}
train_y = [labels[i] for i in train_data['helpfulness']]
test_y = [labels[i] for i in test_data['helpfulness']]

train_x = train_data[['vectorized_text', 'review_time_scl', 'rating_scl', 'richness_scl']]
test_x = test_data[['vectorized_text', 'review_time_scl', 'rating_scl', 'richness_scl']]

Create baseline

In [ ]:
def baseline_model(train_x, train_y):
    model = DummyClassifier(strategy='most_frequent')
    model.fit(train_x, train_y)
    return model

Create evaluation

In [ ]:
def evaluate_model(model, test_x,test_y):
    predict_y = model.predict(test_x)
    print(classification_report(test_y, predict_y, ))

    deciles = pd.DataFrame([(p, t) for p,t in zip(predict_y, test_y)], columns=['Predicted', 'Observed'])
    deciles['Decile'] = pd.qcut(deciles.index, 10, labels=False)
    mean_actual = deciles['Observed'].mean()
    deciles_mean = deciles.groupby(['Decile']).agg(
        Predicted=('Predicted', 'mean'),
        Observed=('Observed', 'mean')).sort_values('Decile', ascending=False)

    deciles_mean['Lift'] = deciles_mean.apply(lambda row: row['Observed']/mean_actual, axis=1)

    tdl = deciles_mean['Lift'].iloc[0] / deciles_mean['Lift'].mean()
    print(f'\nTDL is {tdl}\n')

    gini_auc = roc_auc_score(test_y, predict_y)
    gini = gini_auc*2-1
    print(f'\nGINI is {gini}\n')

Run model and evaluation

In [ ]:
    model = baseline_model(train_x, train_y)
    evaluate_model(model, test_x, test_y)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3164
           1       0.55      1.00      0.71      3855

    accuracy                           0.55      7019
   macro avg       0.27      0.50      0.35      7019
weighted avg       0.30      0.55      0.39      7019


TDL is 1.008935749992878


GINI is 0.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
